# Čtvrté cvičení - vyhlazování jazykového modelu
- Vytvořte bigramový jazykový model vyhlazený metodou Witten-Bell pro soubor TEXTEN1
- Soubor přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase
- Uvažujte na začátku věty startovací symbol <s>

In [18]:
import re, string
from collections import Counter

def text_to_lines(text):
    text_lined = []
    line = ["<s>"]
    for word in text:
        word_stripped = word.strip()
        if word_stripped in (".", "?", "!"):
            text_lined.append(line)
            line = ["<s>"]
        elif word_stripped not in string.punctuation:
            line.append(word_stripped.translate(str.maketrans('', '', string.punctuation)).lower())
    return text_lined

text_en = []
with open('TEXTEN1.txt', 'r', encoding='UTF-8') as file_en:
    text_en = file_en.readlines()

text_en_lined = text_to_lines(text_en)

pairs_counts = Counter()
words_counts = Counter()

for words in text_en_lined:
    words_counts.update(words)
    pairs = [f"{words[i]} {words[i+1]}" for i in range(len(words)-1)]
    pairs_counts.update(pairs)

vocab_size = len(words_counts)

en_bigram_model = {}
for word, _ in words_counts.items():
    N = 1
    T = 1
    possible_next_words = []
    for pair in pairs_counts.keys():
        if pair.startswith(word): 
            N += pairs_counts[pair]
            T += 1
            possible_next_words.append(pair.split()[1])
    all_possible_next_words = words_counts.keys()
    Z = vocab_size - T
    if T == 0:
        T += 1
    if N == 0:
        N += 1
    for next_word in set(all_possible_next_words):
        bigram_str = f"{word} {next_word}"
        c = pairs_counts[bigram_str]

        if c > 0:
            smoothed_count = c / (N + T)
        else:
            smoothed_count = T / (Z * (N + T))

        en_bigram_model[bigram_str] = smoothed_count

print(en_bigram_model["of the"])
# 0.258387066629287
print(en_bigram_model["of of"])
# 1.6608937523330386e-05

0.2430128317806293
1.917485864068185e-05
